In [1]:
# Set up parameters, image paths and category list
# %matplotlib notebook
# %matplotlib widget 
%load_ext autoreload
%autoreload 2

import cv2
import numpy as np
import os.path as osp
import pickle
from random import shuffle
import matplotlib.pyplot as plt
from utils import *
import student_code_12011404 as sc


# This is the list of categories / directories to use. The categories are
# somewhat sorted by similarity so that the confusion matrix looks more
# structured (indoor and then urban and then rural).
categories = ['Kitchen', 'Store', 'Bedroom', 'LivingRoom', 'Office', 'Industrial', 'Suburb',
              'InsideCity', 'TallBuilding', 'Street', 'Highway', 'OpenCountry', 'Coast',
              'Mountain', 'Forest'];
# This list of shortened category names is used later for visualization
abbr_categories = ['Kit', 'Sto', 'Bed', 'Liv', 'Off', 'Ind', 'Sub',
                   'Cty', 'Bld', 'St', 'HW', 'OC', 'Cst',
                   'Mnt', 'For'];

# Number of training examples per category to use. Max is 100. For
# simplicity, we assume this is the number of test cases per category, as
# well.
num_train_per_cat = 100

# This function returns lists containing the file path for each train
# and test image, as well as lists with the label of each train and
# test image. By default all four of these lists will have 1500 elements
# where each element is a string.
data_path = osp.join('..', 'data')
train_image_paths, test_image_paths, train_labels, test_labels = get_image_paths(data_path,
                                                                                 categories,
                                                                                 num_train_per_cat);

In [2]:
import joblib
from joblib import memory
memory = joblib.Memory('./tmp')

In [3]:
build = memory.cache(sc.build_vocabulary_parrallel)
测试写的对不对 = train_image_paths[:64]
build(测试写的对不对, 1).shape
build(测试写的对不对, 24).shape # 看来没问题

________________________________________________________________________________
[Memory] Calling student_code_12011404.build_vocabulary_parrallel...
build_vocabulary_parrallel([ '../data/train/Kitchen/image_0034.jpg',
  '../data/train/Kitchen/image_0174.jpg',
  '../data/train/Kitchen/image_0082.jpg',
  '../data/train/Kitchen/image_0130.jpg',
  '../data/train/Kitchen/image_0083.jpg',
  '../data/train/Kitchen/image_0081.jpg',
  '../data/train/Kitchen/image_0055.jpg',
  '../data/train/Kitchen/image_0144.jpg',
  '../data/train/Kitchen/image_0074.jpg',
  '../data/train/Kitchen/image_0109.jpg',
  '../data/train/Kitchen/image_0076.jpg',
  '../data/train/Kitchen/image_0106.jpg',
  '../data/train/Kitchen/image_0103.jpg',
  '../data/train/Kitchen/image_0026.jpg',
  '../data/train/Kitchen/image_0172.jpg',
  '../data/train/Kitchen/image_0157.jpg',
  '../data/train/Kitchen/ima..., 
1)


100%|██████████| 64/64 [00:02<00:00, 25.27it/s]


_______________________________________build_vocabulary_parrallel - 3.1s, 0.1min
________________________________________________________________________________
[Memory] Calling student_code_12011404.build_vocabulary_parrallel...
build_vocabulary_parrallel([ '../data/train/Kitchen/image_0034.jpg',
  '../data/train/Kitchen/image_0174.jpg',
  '../data/train/Kitchen/image_0082.jpg',
  '../data/train/Kitchen/image_0130.jpg',
  '../data/train/Kitchen/image_0083.jpg',
  '../data/train/Kitchen/image_0081.jpg',
  '../data/train/Kitchen/image_0055.jpg',
  '../data/train/Kitchen/image_0144.jpg',
  '../data/train/Kitchen/image_0074.jpg',
  '../data/train/Kitchen/image_0109.jpg',
  '../data/train/Kitchen/image_0076.jpg',
  '../data/train/Kitchen/image_0106.jpg',
  '../data/train/Kitchen/image_0103.jpg',
  '../data/train/Kitchen/image_0026.jpg',
  '../data/train/Kitchen/image_0172.jpg',
  '../data/train/Kitchen/image_0157.jpg',
  '../data/train/Kitchen/ima..., 
24)


100%|██████████| 64/64 [00:02<00:00, 21.46it/s]

_______________________________________build_vocabulary_parrallel - 3.7s, 0.1min


(24, 128)

In [ ]:
images = train_image_paths+test_image_paths
labels = train_labels+test_labels

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
def evaluate(pipeline, data, target, cross=True):
    X_train, X_test, y_train, y_test = train_test_split(data, target)
    if cross:
        scores = cross_val_score(pipeline, X_train, y_train) # 没有指定estimator，使用模型默认的estimator，就是R2。
        return scores.mean()
    else:
        return pipeline.fit(X_train, y_train).score(X_test, y_test)

evaluate(sc.SiftClassifier(1, ), images, labels, cross=False)

NameError: name 'images' is not defined